### 0. Imports

In [69]:
%load_ext autoreload
%autoreload 2

# Data transformation
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizations
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Progress loops
# -----------------------------------------------------------------------
from tqdm import tqdm

# Handle warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("once")

# modify system variables
# -----------------------------------------------------------------------
import sys
sys.path.append("..") # append parent folder to path

# import support functions
# -----------------------------------------------------------------------
import src.soporte_eda as se
import src.soporte_preprocesamiento as sp
import src.soporte_clustering as sc


# statistics functions
# -----------------------------------------------------------------------
from scipy.stats import pearsonr, spearmanr, pointbiserialr



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Intro to this notebook and data

## 1.1 Introduction

The goal of this busines case is to 1. identify similar groups of clients according to their buying behaviour 2. identify groups of products according to their profitability. For that, analysis, clustering algorithms and regression models will be used.

The purpose of this notebook is to explore Global Ecommerce's product and customers data in order to clean any possible errors that could impair the extraction of insights through analysis and the quality of the subsequent model development for their brand.

## 1.2 Data import

In [70]:
global_superstore = pd.read_csv("../data/Global_Superstore.csv", encoding="latin1")

# transform column names into lowercase and replace spaces by underscores
global_superstore.columns = [col.lower().replace(" ", "_") for col in global_superstore.columns]

global_superstore.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024.0,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,NaN,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,NaN,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


# 2. Preliminary data analysis and cleaning

For a quick initial exploration:

In [71]:
pd.set_option("display.max_columns",24)
se.exploracion_dataframe(global_superstore)

El número de datos es 51290 y el de columnas es 24

 ..................... 

5 filas aleatorias del dataframe son:


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
44646,45406,UP-2012-9910,03-05-2012,07-05-2012,Standard Class,RD-9900,Ruben Dartt,Consumer,Donets'k,Donetsk,Ukraine,NaN,EMEA,EMEA,OFF-ADV-10002593,Office Supplies,Fasteners,"Advantus Clamps, Bulk Pack",19.2900,1,0.000,7.7100,1.30,Medium
2690,9907,MX-2011-104794,04-04-2011,08-04-2011,Standard Class,BW-11065,Barry Weirich,Consumer,Limeira,São Paulo,Brazil,NaN,LATAM,South,TEC-CO-10000388,Technology,Copiers,"Canon Fax Machine, Color",1063.9678,5,0.002,-2.1322,107.66,High
44920,36134,CA-2014-106831,02-06-2014,04-06-2014,First Class,FH-14350,Fred Harton,Consumer,Dublin,Ohio,United States,43017.0,US,East,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980,2,0.700,-2.6586,1.25,Critical
37986,33794,CA-2012-122371,26-09-2012,01-10-2012,Standard Class,BS-11800,Bryan Spruell,Home Office,San Diego,California,United States,92037.0,US,West,OFF-EN-10000056,Office Supplies,Envelopes,Cameo Buff Policy Envelopes,124.4600,2,0.000,58.4962,2.74,Medium
1673,23187,IN-2013-44005,20-08-2013,27-08-2013,Standard Class,CR-12580,Clay Rozendal,Home Office,Ho Chi Minh City,Ho Chí Minh City,Vietnam,NaN,APAC,Southeast Asia,TEC-PH-10003613,Technology,Phones,"Apple Signal Booster, VoIP",1361.9304,12,0.170,49.0104,150.25,Medium



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
row_id,int64,51290
order_id,object,25035
order_date,object,1430
ship_date,object,1464
ship_mode,object,4
customer_id,object,1590
customer_name,object,795
segment,object,3
city,object,3636
state,object,1094



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
product_name,,
Staples,227,0.4
"Cardinal Index Tab, Clear",92,0.2
"Eldon File Cart, Single Width",90,0.2
"Rogers File Cart, Single Width",84,0.2
"Ibico Index Tab, Clear",83,0.2


La columna QUANTITY tiene 14 valores únicos, de los cuales los primeros son:


,count,pct
quantity,,
2,12748,24.9
3,9682,18.9
1,8963,17.5
4,6385,12.4
5,4882,9.5


La columna ORDER_PRIORITY tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
order_priority,,
Medium,29433,57.4
High,15501,30.2
Critical,3932,7.7
Low,2424,4.7


**Aggregation level**

First and foremost, it necessary to notice that the data at hand consists of unaggregated order records data, as there are 51290 rows with 25035 different order ids, which means there can be more than one row per order. 

To use this data to analyse by clients or to analyse by products, agggregation on the appropiate level will be necessary.

### Duplicates

**Row level**

There are 0 duplicates according to the exploration. Let us try by subsets of data:

In [72]:
subset_columns = global_superstore.columns.drop("row_id")
global_superstore.duplicated(subset=subset_columns).sum()

np.int64(0)

0 duplicates are found. Let us observe the duplicated order_ids for instance:

In [73]:
duplicated_mask = global_superstore.duplicated(subset="order_id",keep=False)
global_superstore[duplicated_mask].sort_values(by="order_id").head(4)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
34136,48312,AE-2011-9160,03-10-2011,07-10-2011,Standard Class,PO-8865,Patrick O'Donnell,Consumer,Ajman,'Ajman,United Arab Emirates,NaN,EMEA,EMEA,TEC-EPS-10004171,Technology,Machines,"Epson Calculator, Red",78.408,6,0.7,-88.992,3.87,Medium
29578,48313,AE-2011-9160,03-10-2011,07-10-2011,Standard Class,PO-8865,Patrick O'Donnell,Consumer,Ajman,'Ajman,United Arab Emirates,NaN,EMEA,EMEA,OFF-FEL-10001405,Office Supplies,Storage,"Fellowes File Cart, Industrial",82.674,2,0.7,-157.086,5.69,Medium
5597,47298,AE-2013-1130,14-10-2013,14-10-2013,Same Day,EB-4110,Eugene Barchas,Consumer,Ras al Khaymah,Ra's Al Khaymah,United Arab Emirates,NaN,EMEA,EMEA,FUR-BUS-10003055,Furniture,Bookcases,"Bush Stackable Bookrack, Pine",224.748,6,0.7,-232.272,60.08,High
51156,47297,AE-2013-1130,14-10-2013,14-10-2013,Same Day,EB-4110,Eugene Barchas,Consumer,Ras al Khaymah,Ra's Al Khaymah,United Arab Emirates,NaN,EMEA,EMEA,OFF-ACC-10004278,Office Supplies,Fasteners,"Accos Paper Clips, Bulk Pack",4.248,1,0.7,-4.692,0.10,High


As we suspected, order data is disaggregated per product data.

**Customer id**

Customer_ids also look duplicated, as there are more ids than there are names and exactly double, which is suspicious.

In [74]:
customer_id_name_count = global_superstore[["customer_id",
                                            "customer_name"]].value_counts().reset_index()

# min of duplicated customer name
counts_min = customer_id_name_count["customer_name"].value_counts().min()

print(f"The minimum number of appearances by customer is {counts_min}.")

# leaving this long print for explanatory purposes
customer_id_name_count.sort_values("customer_name").head(16)

The minimum number of appearances by customer is 2.


,customer_id,customer_name,count
1160,AB-15,Aaron Bergman,12
19,AB-10015,Aaron Bergman,77
1199,AH-30,Aaron Hawkins,12
571,AH-10030,Aaron Hawkins,44
1154,AS-45,Aaron Smayling,13
516,AS-10045,Aaron Smayling,47
256,AB-10060,Adam Bellavance,57
1225,AB-60,Adam Bellavance,11
132,AH-10075,Adam Hart,64
883,AH-75,Adam Hart,20


Same customers have 2 different ids. This happens for all customers. From observation it is clear that those ids are in fact the same but with different format. Taking the example below it is obvious that it is not a case of new id formatting, just a typo. Thus we correct it by unifying ids by customer name.

In [78]:
global_superstore[global_superstore["customer_name"] == "Aaron Bergman"].sort_values(by="order_date").head(8)

# generate id mapping
id_mapping = (customer_id_name_count.sort_values("customer_id")
                    .drop_duplicates(subset="customer_name")[["customer_name","customer_id"]]
                    .set_index("customer_name")
                    .to_dict()["customer_id"])

global_superstore["customer_id"] = global_superstore["customer_name"].map(id_mapping)

**Product_id**

From the unique value counts of the above exploration, we can see that there are 10292 product ids although there only 3788 product names. Given that there are no null values for those columns, it is necessary to explore why:

In [87]:
global_superstore.groupby("product_id")["product_name"].count().describe()

count    10292.000000
mean         4.983482
std          3.410633
min          1.000000
25%          2.000000
50%          4.000000
75%          7.000000
max         35.000000
Name: product_name, dtype: float64

At least a 75% of products have 2 or more ids. Let us inspect the specific values for each.

In [92]:
product_id_name_count = global_superstore[["product_id",
                                            "product_name"]].value_counts().reset_index()

# leaving this long print for explanatory purposes
product_id_name_count.shape

(10768, 3)

There are 10768 combinations of product name and product id, which is more than there are product_ids, which means not only do products have more than 1 id, but also ids have more than 1 name.

Therefore, the approach is to create new ids for each product, grouping by their name.

In [106]:
# generate new ids
unique_product_names = global_superstore["product_name"].unique()
produc_ids = [f"PR-{id}" for id in range(0,len(unique_product_names))]

# generate new product id mapping
id_mapping = dict(zip(unique_product_names, produc_ids))

# apply changes
global_superstore["product_id"] = global_superstore["product_name"].map(id_mapping)

Now checking the change's results:

In [107]:
# checking changes
global_superstore[["product_id","product_name"]].nunique()

product_id      3788
product_name    3788
dtype: int64

#### Missing values

Drop postal_code as it adds no value due to having 80% of missing.

In [110]:
global_superstore.drop(columns="postal_code", inplace=True, errors="ignore")

#### Numerical features ranges

Sales are not integers because it actually makes reference to the product revenue, where profit would be the profit for that specific product for that order. 

Profits can be negative, which makes it very interesting to control which orders provide negative profits, as a means to minimize this loss.

Ranges for discounts are reasonable.

Shipping costs can be get very high. It is interesting to see if shipping costs are always below the sales amount:

In [112]:
(global_superstore["shipping_cost"] > global_superstore["sales"]).sum()

np.int64(0)

Then we can conclude that shipping costs are assumed by the customer.

The percentage of costs from the sale due to shipping follows the below distribution:

In [114]:
(global_superstore["shipping_cost"] / global_superstore["sales"]).describe()

count    51290.000000
mean         0.109226
std          0.069933
min          0.000000
25%          0.063136
50%          0.090254
75%          0.136364
max          0.589638
dtype: float64

#### Categorical features

#### Other checks

**Countries and regions per client**

Judging from excerpts of data, it looks like customers either order from or to many different countries.

In [76]:
global_superstore.groupby("customer_id")[["country"]].nunique().describe()

,country
count,795.000000
mean,19.748428
std,3.376778
min,8.000000
25%,17.000000
50%,20.000000
75%,22.000000
max,32.000000


The above describe shows that the minimum number of countries for a customer is 8, which is not impossible but not plausible. Let us make this check with a customers' region.

In [77]:
global_superstore.groupby("customer_id")[["region"]].nunique().describe()

,region
count,795.000000
mean,10.661635
std,1.164104
min,6.000000
25%,10.000000
50%,11.000000
75%,11.000000
max,13.000000


More than 75% of the customers ship to or order from at least 10 different countries, being 6 the minimum, which is very difficult to believe.

If we look at the most frequent country distribution, we see that there is no majority orders from a certain country.

In [84]:
# get most frequent country per customer 
most_frequent = global_superstore.groupby('customer_id')['country'].apply(
    lambda x: x.value_counts().idxmax()
)

# get the relative frequency for the top country
frequency = global_superstore.groupby('customer_id')['country'].apply(
    lambda x: x.value_counts().max() / x.count()
)

# get most frequent's country frequency distribution
result = pd.DataFrame({'most_frequent_country': most_frequent, 'frequency': frequency})
result.describe()

,frequency
count,795.000000
mean,0.210975
std,0.072899
min,0.079365
25%,0.156250
50%,0.200000
75%,0.250000
max,0.472222


Therefore, city, state, country and region are of no further use in this project, as it is very impractical to use this data for clustering or regression.

In [85]:
global_superstore.drop(columns=["region","country","city","state"], inplace=True, errors="ignore")